In [55]:
import pandas as pd
import random
from datetime import timedelta

#Planned Order Release Date sütununu datetime formatına çevirelim
df_stock["Planned Order Release Date"] = pd.to_datetime(df_stock["Planned Order Release Date"], errors="coerce")

#Kategorilere göre raf ömrü (gün cinsinden)
expiration_rules = {
    "BEBEK": (180, 730),
    "DETERJAN TEMİZLİK": (365, 1825),
    "ET TAVUK": (2, 7),
    "EV": (365, 1825),
    "GIDA": (30, 365),
    "KAĞIT": (365, 1825),
    "KOZMETİK": (180, 730),
    "MEYVE SEBZE": (3, 14),
    "PET": (90, 365),
    "SÜT KAHVALTILIK": (7, 30),
    "SİGARA": (365, 1825),
    "İÇECEK": (90, 365)
}

# Son tüketim tarihini hesaplayan fonksiyon
def calculate_expiration_date(row):
    category = row.get("CATEGORY_NAME1", "")

    # 📌 Eğer kategori boş veya NaN ise, boş string olarak ata
    if pd.isna(category) or not isinstance(category, str):
        return None  # 🔹 Eğer kategori yoksa SKT hesaplanmayacak

    category = category.strip().upper()  # 🔹 Kategoriyi normalize et
    stock_date = row["Planned Order Release Date"]  # 🔹 Stoğa giriş tarihi

    if pd.isna(stock_date):  # 🔹 Eğer stoğa giriş tarihi boşsa SKT de boş olacak
        return None  

    # Kategoriye göre rastgele raf ömrü belirleme
    min_days, max_days = expiration_rules.get(category, (30, 365))  # 🔹 Varsayılan 30-365 gün
    expiration_date = stock_date + timedelta(days=random.randint(min_days, max_days))

    return expiration_date.strftime("%Y-%m-%d")

#SKT'yi hesaplayıp yeni sütun olarak ekleyelim
df_stock["expiration_date"] = df_stock.apply(calculate_expiration_date, axis=1)

#Güncellenmiş stok verisini tekrar kaydedelim
#df.to_csv("ime_market/Stock_with_SKT.csv", index=False)

# İlk birkaç satırı görüntüleyelim
print(df_stock.head())


   MagzaID  ITEMCODE  Week  Year  Weekly Demand  Projected On Hand  \
0        5      5863     1  2024            0.0                  0   
1        5      5863     1  2024            0.0                  0   
2        5      5863     1  2024            0.0                  0   
3        5      5863     2  2024            0.0                  0   
4        5      5863     2  2024            0.0                  0   

   Net Requirements  Planned Order Receipts  Planned Order Releases  \
0               0.0                     0.0                     0.0   
1               0.0                     0.0                     0.0   
2               0.0                     0.0                     0.0   
3               0.0                     0.0                     0.0   
4               0.0                     0.0                     0.0   

  Planned Order Receipt Date Planned Order Release Date CATEGORY_NAME1  \
0                        NaN                        NaT         İÇECEK   
1   

In [65]:
aa=df_stock.drop(columns=["CATEGORY_NAME1"])

In [71]:
aa.to_csv("skt.csv",index=False)

In [51]:
df_stock.merge(df_category, on="ITEMCODE")

,MagzaID,ITEMCODE,Week,Year,Weekly Demand,Projected On Hand,Net Requirements,Planned Order Receipts,Planned Order Releases,Planned Order Receipt Date,Planned Order Release Date,CATEGORY_NAME1_x,CATEGORY_NAME1_y
0,5,5863,1,2024,0.0,0,0.0,0.0,0.0,NaN,NaT,İÇECEK,İÇECEK
1,5,5863,1,2024,0.0,0,0.0,0.0,0.0,NaN,NaT,İÇECEK,İÇECEK
2,5,5863,1,2024,0.0,0,0.0,0.0,0.0,NaN,NaT,İÇECEK,İÇECEK
3,5,5863,1,2024,0.0,0,0.0,0.0,0.0,NaN,NaT,İÇECEK,İÇECEK
4,5,5863,1,2024,0.0,0,0.0,0.0,0.0,NaN,NaT,İÇECEK,İÇECEK
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103722679,5,23389,48,2024,1.0,0,1.0,1.0,1.0,2024-11-25,2024-11-18,BEBEK,BEBEK
103722680,5,23389,49,2024,0.0,0,0.0,0.0,0.0,NaN,NaT,BEBEK,BEBEK
103722681,5,23389,50,2024,0.0,0,0.0,0.0,0.0,NaN,NaT,BEBEK,BEBEK
103722682,5,23389,51,2024,0.0,0,0.0,0.0,0.0,NaN,NaT,BEBEK,BEBEK


In [21]:
import pandas as pd
import random
from datetime import datetime, timedelta

# 📌 Dosya yollarını belirleyin
stock_file_path = "ime_market/Stock.csv"
category_file_path = "ime_market/aaaaaa_nnew.xlsx"

# 📌 Stock.csv'yi yükleyelim (Stoğa geliş tarihleri için)
df_stock = pd.read_csv(stock_file_path, usecols=["ITEMCODE", "Planned Order Release Date"])

# 📌 Planned Order Release Date sütununu datetime formatına çevirelim
df_stock["Planned Order Release Date"] = pd.to_datetime(df_stock["Planned Order Release Date"], errors="coerce")

# 📌 aaaaaa.xlsx dosyasından sadece ITEMCODE ve CATEGORY_NAME1 sütunlarını alarak yükleyelim (İlk 10.000 satır)
df_category = pd.read_excel(category_file_path, sheet_name="Sheet1", usecols=["ITEMCODE", "CATEGORY_NAME1"], nrows=10000)

# 📌 Stock ve kategori bilgilerini ITEMCODE üzerinden birleştirelim
df_merged = df_stock.merge(df_category, on="ITEMCODE", how="left")

# 📌 SADECE "Planned Order Release Date" sütunu DOLU olanları alalım
df_valid_dates = df_merged.dropna(subset=["Planned Order Release Date"]).copy()

# 📌 Kategorilere göre raf ömrü (gün cinsinden)
expiration_rules = {
    "BEBEK": (180, 730),
    "DETERJAN TEMİZLİK": (365, 1825),
    "ET TAVUK": (2, 7),
    "EV": (365, 1825),
    "GIDA": (30, 365),
    "KAĞIT": (365, 1825),
    "KOZMETİK": (180, 730),
    "MEYVE SEBZE": (3, 14),
    "PET": (90, 365),
    "SÜT KAHVALTILIK": (7, 30),
    "SİGARA": (365, 1825),
    "İÇECEK": (90, 365)
}

# 📌 Stoğa geliş tarihine göre son kullanma tarihini hesaplayan fonksiyon
def calculate_expiration_date(row):
    category = row.get("CATEGORY_NAME1", "")

    # 📌 Eğer kategori NaN veya sayısalsa, boş string olarak ata
    if pd.isna(category) or not isinstance(category, str):
        category = ""

    category = category.strip().upper()  # Normalize et
    stock_date = row["Planned Order Release Date"]

    # 📌 Kategori bazlı raf ömrü belirleme
    min_days, max_days = expiration_rules.get(category, (30, 365))  # Varsayılan 30-365 gün
    expiration_date = stock_date + timedelta(days=random.randint(min_days, max_days))

    return expiration_date.strftime("%Y-%m-%d")

# 📌 SADECE TARİHİ OLANLARA SKT ATAMA YAPALIM
df_valid_dates["expiration_date"] = df_valid_dates.apply(calculate_expiration_date, axis=1)

# 📌 Güncellenmiş tabloyu CSV olarak kaydet
df_valid_dates.to_csv("updated_stock_with_skt.csv", index=False)

# 📌 İlk birkaç satırı görüntüle
print(df_valid_dates.head())



    ITEMCODE Planned Order Release Date CATEGORY_NAME1 expiration_date
18      5863                 2024-02-05         İÇECEK      2024-09-14
19      5863                 2024-02-05         İÇECEK      2024-08-18
20      5863                 2024-02-05         İÇECEK      2024-10-20
24      5863                 2024-02-19         İÇECEK      2024-06-29
25      5863                 2024-02-19         İÇECEK      2024-11-08


In [49]:
df_merged

,ITEMCODE,Planned Order Release Date,CATEGORY_NAME1
0,5863,NaN,İÇECEK
1,5863,NaN,İÇECEK
2,5863,NaN,İÇECEK
3,5863,NaN,İÇECEK
4,5863,NaN,İÇECEK
...,...,...,...
4844783,14305,NaN,NaN
4844784,14305,NaN,NaN
4844785,14305,NaN,NaN
4844786,14305,NaN,NaN


In [ ]:
# Yeni veri seti için satır sayısını belirleme
new_rows = len(df) * 2

# 2024 yılı içinde rastgele tarih atama
random_dates = pd.date_range(start="2024-01-01", end="2024-12-31", periods=new_rows).strftime('%Y-%m-%d')

# ITEMCODE değerlerini mevcut veri setinden rastgele seçme
random_itemcodes = np.random.choice(df["ITEMCODE"].unique(), new_rows, replace=True)

# PRICE ve AMOUNT değerlerini çoğaltma
random_prices = np.random.normal(df["PRICE"].mean(), df["PRICE"].std(), new_rows)
random_prices = np.clip(random_prices, df["PRICE"].min(), df["PRICE"].max())

random_amounts = np.random.normal(df["AMOUNT"].mean(), df["AMOUNT"].std(), new_rows)
random_amounts = np.clip(random_amounts, df["AMOUNT"].min(), df["AMOUNT"].max())

# Yeni veri setini oluşturma
new_df = pd.DataFrame({
    "DATE": random_dates,
    "ITEMCODE": random_itemcodes,
    "AMOUNT": random_amounts,
    "PRICE": random_prices
})